In [1]:
!wget -O Train.npy https://www.dropbox.com/s/sn8v6077bi8z1a6/Train.npy?dl=0

--2019-01-20 03:23:16--  https://www.dropbox.com/s/sn8v6077bi8z1a6/Train.npy?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/sn8v6077bi8z1a6/Train.npy [following]
--2019-01-20 03:23:22--  https://www.dropbox.com/s/raw/sn8v6077bi8z1a6/Train.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6dc10579c59da84c7dbc3708aa.dl.dropboxusercontent.com/cd/0/inline/AZsQNcD91c_i0q2bQGUYewivJHX2V7nOaeN4IHYkT0ObtGpzReUk4u7kFpu_wLrsFlhjBqfyMs3-S4cxEMqryh1THE91bHskHSGoH46e2wWxBQ/file# [following]
--2019-01-20 03:23:22--  https://uc6dc10579c59da84c7dbc3708aa.dl.dropboxusercontent.com/cd/0/inline/AZsQNcD91c_i0q2bQGUYewivJHX2V7nOaeN4IHYkT0ObtGpzReUk4u7kFpu_wLrsFlhjBqfyMs3-S4cxEMqryh1THE91bHskHSGoH46e2wWxBQ/file
Resolving uc6dc10579

In [2]:
ls -l --block-size=M

total 112M
drwxr-xr-x 1 root root   1M Jan  8 17:15 sample_data/
-rw-r--r-- 1 root root 112M Jan 20 03:23 Train.npy


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
Data=np.load('Train.npy')
print (Data.shape)
X=Data[:,np.arange(Data.shape[1])!=468]
y=Data[:,468]
print (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

(61566, 473)
[0. 0. 0. ... 5. 5. 5.]


In [4]:
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import Model
from keras.losses import categorical_crossentropy
import numpy as np
from keras.optimizers import Adam
from keras.utils import to_categorical
class DNN:
    """
    Class to construct Fully Connected Deep Networks
    """
    def __init__(self, config):
        """

        :param config: Dictionary of network configurations
        """
        self.input_shape = config['Input_shape']
        self.layer_shapes = config['layer_shapes']
        self.activation = config['Activation']
        self.output = config['Output']
        self.weights=config['Weights']
        self.config=config
        self.model=self.construct_fc_model()
        self.compile()
        if self.weights:
            self.load_weights()

    def construct_fc_model(self):
        """
        Constructs a Fully Connected model
        :return: Model object of keras
        """
        input_layer = Input(shape=(self.input_shape,))
        layer = input_layer
        layer = BatchNormalization()(layer)
        for layer_neurons in self.layer_shapes:
            layer = Dense(layer_neurons)(layer)
            layer=Activation('relu')(layer)
            layer = BatchNormalization()(layer)
            layer = Dropout(self.config['dropout'])(layer)
        layer = Dense(self.output,activation='softmax')(layer)
        return Model(input_layer, layer)
      
    def load_weights(self):
        """
        Load weights into a model
        :return: Nothing
        """
        self.model.load_weights(self.weights)

    def predict(self, input):
        """
        Returns the predicted label
        """
        return self.model.predict(np.atleast_2d(input))
    def compile(self):
      self.model.compile(loss=categorical_crossentropy,optimizer=Adam(0.001,decay=1e-4),metrics=['acc'])

Using TensorFlow backend.


In [0]:
nn_config={'layer_shapes':[500,500,500],'Activation':'relu','Output':6,
           'Input_shape':472,'dropout':0.2,'Weights':None}
Network=DNN(nn_config)

In [33]:
Network.model.fit(X_train,to_categorical(y_train),epochs=800,batch_size=len(X_train),verbose=0)

In [34]:
Network.model.evaluate(X_train,to_categorical(y_train),batch_size=len(X_train))

49252/49252 [==============================] - 0s 10us/step


[0.029831910505890846, 0.9868837594985962]

In [0]:
y_pred=Network.model.predict(X_test).argmax(axis=-1)

In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print ('F1_score',f1_score(y_true=y_test,y_pred=y_pred,average='macro'))
print (confusion_matrix(y_true=y_test,y_pred=y_pred))
print (accuracy_score(y_true=y_test,y_pred=y_pred))

F1_score 0.8601157532652438
[[1865    8   61   74    0    0]
 [  15 1837   84   60   23   37]
 [  66   49 1564  111  134  110]
 [  66   55  112 1583  151  136]
 [   0   21   91   88 1882   30]
 [   0   19   50   59    9 1864]]
0.8604027935682963


In [20]:
from sklearn.ensemble import RandomForestClassifier
R=RandomForestClassifier()
R.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
y_pred=R.predict(X_train)
print ('F1_score',f1_score(y_true=y_train,y_pred=y_pred,average='macro'))
print (confusion_matrix(y_true=y_train,y_pred=y_pred))
print (accuracy_score(y_true=y_train,y_pred=y_pred))

F1_score 0.9927719359928268
[[8242    0    4    7    0    0]
 [   6 8186    3    5    2    3]
 [   9    7 8169    7   23   12]
 [  17   11   12 7999   59   60]
 [   0    3   12   39 8094    1]
 [   0    6    5   28   14 8207]]
0.9927921708763096


In [25]:
y_pred=R.predict(X_test)
print ('F1_score',f1_score(y_true=y_test,y_pred=y_pred,average='macro'))
print (confusion_matrix(y_true=y_test,y_pred=y_pred))
print (accuracy_score(y_true=y_test,y_pred=y_pred))

F1_score 0.8362532226193243
[[1928    1   52   27    0    0]
 [  23 1839   51   62   41   40]
 [  73   86 1495  114  146  120]
 [  81  130  162 1398  177  155]
 [   1   36   96  100 1851   28]
 [   0   36   61   59   34 1811]]
0.8382329056358616
